In [21]:
# install transformers
!pip install transformers --no-build-isolation
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
import os

os.getcwd()
os.listdir()

['test', 'multimodal']

In [23]:
import torch
print(torch.cuda.is_available())  # 應該回傳 True
print(torch.version.cuda)  # 應該顯示 CUDA 版本

True
12.6


In [28]:
from datasets import Dataset, Image
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
dataset = Dataset.from_dict({"image": ["/workspace/multimodal/225626.jpg", "/workspace/multimodal/bee.png"]}).cast_column("image", Image())
image1 = dataset[0]["image"]
# image2 = dataset[1]["image"]


In [29]:
from transformers import AutoProcessor, AutoModelForVision2Seq
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
import torch
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-Instruct",
    torch_dtype=torch.bfloat16
).to("cuda")

In [36]:
# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            # {"type": "image"},
            {"type": "text", "text": "Can you describe the image?"},
        ]
    },
]
# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = inputs.to(DEVICE)
# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)
print(generated_texts[0])


User:<image>Can you describe the image?
Assistant: The image depicts a subway or train station with a long, narrow track running through the center. The track is made of metal and is surrounded by a layer of gravel. The track is bordered by concrete walls on both sides, with a few metal poles and chains visible along the edges. The walls are painted in a light gray color, and the ceiling is made of concrete with exposed pipes and lights.

In the foreground, a man is standing on the track, holding a cardboard box. The man is wearing a black shirt and dark pants, and he has a serious expression on his face. He is standing on the track, which is made of metal and is surrounded by gravel. The man is holding a cardboard box, which is open and contains some items.

To the left of the man, there is a metal railing, which is painted in a bright yellow color. The railing is attached to the concrete wall and is used to prevent people from falling off the track. There is also a yellow caution sig

In [27]:
"""
User:<image>Can you describe the two images?
Assistant: I can describe the first one, but I can't describe the second one.
"""

"\nUser:<image>Can you describe the two images?\nAssistant: I can describe the first one, but I can't describe the second one.\n"